# HuggingFace LLM inference experiments

- Compare inference options for quantisation and flash-atten
- As of 01/2024 torch's built in sdpa attention impmenentation has flash-attention without the need for the external lib from Tri Dao (which can be a pain to install). But it's not currently implemented within HF for all models -- particularly there's no support for Mistral
- So still need to use flash-atten lib for now to get flash-attention v2

In [1]:
import json
import os
import sys
import time
from typing import List, Dict, Any

In [2]:
sys.path.append(os.getcwd())

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

First just load Mistral model with no additional options -- would be just as easy to use their reference implementation but HF gives all the other options for quant etc so go with this

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

device = "cuda" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", torch_dtype=torch.float16, device_map=device)
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|████████████████████████████████████████| 3/3 [00:06<00:00,  2.31s/it]


In [5]:
prompt = "[INST] What is your favourite condiment? [/INST] "
inputs = tokenizer(prompt, return_tensors="pt").to(device)
print(inputs)
generated_ids = model.generate(**inputs, max_new_tokens=256, do_sample=True)
print(generated_ids)
outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
print(outputs)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'input_ids': tensor([[    1,   733, 16289, 28793,  1824,   349,   574, 16020,  2076,  2487,
         28804,   733, 28748, 16289, 28793, 28705]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}
tensor([[    1,   733, 16289, 28793,  1824,   349,   574, 16020,  2076,  2487,
         28804,   733, 28748, 16289, 28793, 28705,   315,   949, 28742, 28707,
           506,   264,  9230,  5369, 28713,   442,   264, 21448,   354,  2076,
          8447, 28723,  2993, 28725,   315,   541,  1912,   368,   369,   741,
          3298, 10392,  3387,  3024,   446,  4455,   715, 28725,  1580,   488,
         28725,   993,  7136,   864, 28725,  3296, 16042, 28725,   579, 28724,
         16042, 28725,   304, 26678, 21993, 28723,   661, 12282,  9618,   356,
          3327,  9230, 22731,   304,   272,  1212,   302,  2887,  1250, 23440,
         28723,     2]], device='cuda:0')
["[INST] What is your favourite condiment? [/INST]  I don't have a 

In [7]:
messages = [
    {"role": "user", "content": "How were the samples in the sp1200 encoded?"},
    {"role": "assistant", "content": "The samples were encoded in 12-bit PCM."},
    {"role": "user", "content": "What could be done to improve the drum sounds?"},
]

In [8]:
# tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")
prompt = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)
inputs = tokenizer(prompt, return_tensors="pt").to(device)
inputs

{'input_ids': tensor([[    1,     1,   733, 16289, 28793,  1602,   654,   272, 11714,   297,
           272,   668, 28740, 28750, 28734, 28734, 23174, 28804,   733, 28748,
         16289, 28793,  1014, 11714,   654, 23174,   297, 28705, 28740, 28750,
         28733,  2581,  9596, 28755, 28723,     2,   733, 16289, 28793,  1824,
           829,   347,  2203,   298,  4916,   272, 16049,  7258, 28804,   733,
         28748, 16289, 28793]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1]], device='cuda:0')}

In [11]:
# encodeds = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)
encodeds = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(device)
print(encodeds)
generated_ids = model.generate(encodeds, max_new_tokens=256, do_sample=True)
outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
print(outputs[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


tensor([[    1,   733, 16289, 28793,  1602,   654,   272, 11714,   297,   272,
           668, 28740, 28750, 28734, 28734, 23174, 28804,   733, 28748, 16289,
         28793,  1014, 11714,   654, 23174,   297, 28705, 28740, 28750, 28733,
          2581,  9596, 28755, 28723,     2,   733, 16289, 28793,  1824,   829,
           347,  2203,   298,  4916,   272, 16049,  7258, 28804,   733, 28748,
         16289, 28793]], device='cuda:0')
[INST] How were the samples in the sp1200 encoded? [/INST]The samples were encoded in 12-bit PCM. [INST] What could be done to improve the drum sounds? [/INST]There are several ways to improve the drum sounds from an SP-1200 or any other sampler using similar technology:

1. Updating or replacing the sampler's internal samples: The drum sounds that came with the SP-1200 were recorded in the late 1980s and early 1990s, so updating or replacing them with modern sample libraries or recordings of higher quality could significantly improve the overall sound.
2. 

same as above but without direct tokenization using the chat template...does make a minor difference to the output but can be more flexible

In [12]:
prompt = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)
inputs = tokenizer(prompt, return_tensors="pt").to(device)
print(inputs)
generated_ids = model.generate(**inputs, max_new_tokens=256, do_sample=True)
outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
print(outputs[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'input_ids': tensor([[    1,     1,   733, 16289, 28793,  1602,   654,   272, 11714,   297,
           272,   668, 28740, 28750, 28734, 28734, 23174, 28804,   733, 28748,
         16289, 28793,  1014, 11714,   654, 23174,   297, 28705, 28740, 28750,
         28733,  2581,  9596, 28755, 28723,     2,   733, 16289, 28793,  1824,
           829,   347,  2203,   298,  4916,   272, 16049,  7258, 28804,   733,
         28748, 16289, 28793]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1]], device='cuda:0')}
[INST] How were the samples in the sp1200 encoded? [/INST]The samples were encoded in 12-bit PCM. [INST] What could be done to improve the drum sounds? [/INST]There are several ways to improve drum sounds in a digital audio workstation (DAW) or a drum machine like the SP-1200:

1. Sampling higher-quality sources: Us

In [16]:
%timeit -r 2 model.generate(**inputs, max_new_tokens=256, do_sample=True)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


16.2 s ± 304 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


In [33]:
import time

n_repeats = 2
total_duration = 0
total_tps = 0
for _ in range(n_repeats):
    start = time.time()
    generated_ids = model.generate(**inputs, max_new_tokens=256, do_sample=True)
    end = time.time()
    duration = end - start
    total_duration += duration
    total_tps += len(generated_ids[0]) / duration
    print(f"duration: {duration:.2f}, t/s: {len(generated_ids[0]) / duration:.2f}")
print(f"Averages for prompt and generated | duration: {total_duration / n_repeats:.2f}, t/s: {total_tps / n_repeats:.2f}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


duration: 16.44 t/s: 18.80
duration: 16.49 t/s: 18.74
Averages for prompt and generated | duration: 16.47 t/s: 18.77


In [ ]:
# Note: this ignores the difference between prompt and generated tokens, but as long as we're comparing like for like
# it's good enough for these purposes. Should find the time for each part separately for a more accurate comparison.
def calculate_average_duration(inputs, model, n_repeats=2, max_new_tokens=256):
    total_duration = 0
    total_tps = 0
    for _ in range(n_repeats):
        start = time.time()
        generated_ids = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=True)
        end = time.time()
        duration = end - start
        total_duration += duration
        total_tps += len(generated_ids[0]) / duration
        print(f"duration: {duration:.2f}, t/s: {len(generated_ids[0]) / duration:.2f}")
    print(f"Averages for prompt and generated | duration: {total_duration / n_repeats:.2f}, t/s: {total_tps / n_repeats:.2f}")

calculate_average_duration(inputs, model)

Quick test for 2 repeats on the inputs defined above gives:

- duration: 16.44 t/s: 18.80
- duration: 16.49 t/s: 18.74
- Averages for prompt and generated | duration: 16.47 t/s: 18.77


## Flash Attention v2

In [34]:
# check that the variable `model` exists and if it does, delete it
if "model" in locals():
    model = model.to("cpu")
    del model
    torch.cuda.empty_cache()

model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.2",
    device_map=device,
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
)

Loading checkpoint shards: 100%|████████████████████████████████████████| 3/3 [00:04<00:00,  1.62s/it]


In [36]:
generated_ids = model.generate(**inputs, max_new_tokens=256, do_sample=True)
outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
print(outputs[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] How were the samples in the sp1200 encoded? [/INST]The samples were encoded in 12-bit PCM. [INST] What could be done to improve the drum sounds? [/INST]There are several ways to improve drum sounds from an Akai S1200 or any other sampler, especially when working with older drum sounds:

1. Re-sampling: You can re-sample the original drum sounds with higher-quality equipment or software, allowing you to maintain the characteristic sound while improving the overall clarity and resolution.

2. Editing: Use an audio editor (such as Adobe Audition, Reaper, or Audacity) to edit the drum sounds. Remove unwanted noise, adjust volume levels, and equalize the EQ to improve the overall tone and balance of the drums.

3. Layering: Layer multiple sounds or samples of the same drum sound to create a fuller, more complex drum hit.

4. Compression: Use a compressor to even out the dynamic range and add punch to the drums.

5. Reverb: Apply a room or ambient reverb effect to the drums to enhance

In [38]:
calculate_average_duration(inputs, model)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
